In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import csv
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier

def clearData(data):
    # First changin ? with orderDate + Mean
    clearedDataFirst = [list(d) if list(d)[1] != "?" else [list(d)[0],(datetime.strptime(list(d)[0], '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d") ]  for d in zip(data['orderDate'], data['deliveryDate']) ]
    # Second changing 1990 with orderDate + Mean
    clearedTimeDataSecond = [d if datetime.strptime(list(d)[0], '%Y-%m-%d') <= datetime.strptime(list(d)[1], '%Y-%m-%d') else [d[0],(datetime.strptime(list(d)[0], '%Y-%m-%d') + timedelta(days=1)).strftime("%Y-%m-%d")] for d in clearedDataFirst]
    data["deliveryDate"] = [row[1] for row in clearedTimeDataSecond]

    # Clearing the birthdate data
    allDataFirstAgeChanged = [d if d != "?" else (datetime.strptime("2014-01-01", '%Y-%m-%d') - timedelta(days=48)).strftime('%Y-%m-%d') for d in data['dateOfBirth']   ]
    allDataAllAgeChanged = [d if datetime.strptime(d, '%Y-%m-%d') > datetime.strptime("1920-01-01", '%Y-%m-%d') and datetime.strptime(d, '%Y-%m-%d') < datetime.strptime("2000-01-01", '%Y-%m-%d') else (datetime.strptime("2014-01-01", '%Y-%m-%d') - timedelta(days=30*365.2)).strftime('%Y-%m-%d') for d in allDataFirstAgeChanged]
    data["dateOfBirth"] = allDataAllAgeChanged
    
    return data



def createFeautre(data, rDict, pDict, mDict, cDcit):
    # Feature 1, if taken date > 10 1, else 0
    shipmentDateF = np.array([math.floor((datetime.strptime(list(d)[1], '%Y-%m-%d')-datetime.strptime(list(d)[0], '%Y-%m-%d')) / timedelta (days=1)) for d in zip(data['orderDate'], data['deliveryDate'])])

    # Feature 2 - specialDateChrismass, if order is close to speacial date and came late it is 1
    # Feature 3 - specialDateOthers, if order is close to speacial date and came late it is 1
    dateRepresentationOfData = [ [(int('%02d' % datetime.strptime(list(d)[0], '%Y-%m-%d').day),int('%02d' % datetime.strptime(list(d)[0], '%Y-%m-%d').month)), (int('%02d' % datetime.strptime(list(d)[1], '%Y-%m-%d').day),int('%02d' % datetime.strptime(list(d)[1], '%Y-%m-%d').month))] for d in zip(data['orderDate'], data['deliveryDate'])]
    specialDateChristmasF = np.array([1 if (d[0][0] > 14 and d[0][0] < 22 and d[0][1] == 12) and (d[1][0] >= 1 and d[1][1] >= 1) else 0 for d in dateRepresentationOfData])
    specialDateOthersF = np.array([1 if (d[0][0] > 3 and d[0][0] < 11 and d[0][1] == 5) and (d[1][0] >= 15 and d[1][1] >= 5) or (d[0][0] > 8 and d[0][0] < 14 and d[0][1] == 2) and (d[1][0] >= 18 and d[1][1] >= 2) else 0 for d in dateRepresentationOfData])

    # Feature 4 
    ageNummArray = [((datetime.strptime("2014-01-01", '%Y-%m-%d')-datetime.strptime(d, '%Y-%m-%d'))/timedelta (days=1))/365 for d in data['dateOfBirth']]
    ageNumF = np.array([ 0 if d < 30 else 1 for d in ageNummArray])
    
    # Feature 5
    accountAge = [((datetime.strptime("2014-01-01", '%Y-%m-%d')-datetime.strptime(d, '%Y-%m-%d'))/timedelta (days=1))/365 for d in data["creationDate"]]
    
    accountAgeF = np.array([ 0 if d < 360 else 1 for d in ageNummArray])
    
    # Feature 6 manufacturer id mean value
    manuReturnRateF = np.array([(mDict[str(d) + "-1"] + 5)/(mDict[str(d) + "-1"] + 10 + mDict[str(d) + "-0"] ) for d in data['manufacturerID'] ])


    # Feature 7 costumer id mean value
    userReturnRateF = np.array([(cDcit[str(d) + "-1"] + 5)/(cDcit[str(d) + "-1"] + 10 + cDcit[str(d) + "-0"] ) for d in data['customerID'] ])


    # Feature 8 risk
    rZeroF = np.array([(rDict[str(d) + "-1"] + 5)/(rDict[str(d) + "-1"] + 10 + rDict[str(d) + "-0"] ) for d in data['itemID'] ])
    
    # Feature 9 price
    priceF = np.array([list(d)[1]/pDict[str(list(d)[0])] for d in zip(data['itemID'],data['price'])])

    

            
        
    
    return  np.vstack((shipmentDateF, specialDateChristmasF, specialDateOthersF, ageNumF, accountAgeF, manuReturnRateF, userReturnRateF, rZeroF, priceF)).T
    

In [2]:
data = clearData(pd.read_csv('train.txt', sep=","))

In [3]:
rDict = {}
for i in range(3705):
    rDict[str(i) + "-0"] = 0
    rDict[str(i) + "-1"] = 0
grouped_df = data.groupby(['itemID', 'returnShipment'])['itemID', 'returnShipment']
for key, item in grouped_df:
    rDict[str(key[0]) + "-" + str(key[1])] = len(grouped_df.get_group(key))
    
    
pDict = {}
for i in range(4000):
    pDict[str(i)] = (1, 1)    
grouped_df = data.groupby(['itemID', 'price'])['itemID', 'price']

for key, item in grouped_df:
    pDict[str(key[0])] = (pDict[str(key[0])][0]+key[1], pDict[str(key[0])][1] + 1)


for key, item in pDict.items():
    if item[1] == 1:
        pDict[key] = item[0]
    if item[1] != 1:
        pDict[key] = item[0]/(item[1]-1)
    

mDict = {}
for i in range(200):
    mDict[str(i) + "-0"] = 0
    mDict[str(i) + "-1"] = 0
grouped_df = data.groupby(['manufacturerID', 'returnShipment'])['manufacturerID', 'returnShipment']
for key, item in grouped_df:
    mDict[str(key[0]) + "-" + str(key[1])] = len(grouped_df.get_group(key))
    
    
cDict = {}
for i in range(92000):
    cDict[str(i) + "-0"] = 0
    cDict[str(i) + "-1"] = 0
grouped_df = data.groupby(['customerID', 'returnShipment'])['customerID', 'returnShipment']
for key, item in grouped_df:
    cDict[str(key[0]) + "-" + str(key[1])] = len(grouped_df.get_group(key))

In [4]:
# Creating train matrix
trainMatrix = createFeautre(data, rDict, pDict, mDict, cDict)

# Creating label array
labelArray = np.array(data['returnShipment'])


In [5]:
testData =  clearData( pd.read_csv('test.txt', sep=","))

In [6]:
rDictTest = {}
for i in range(4000):
    rDictTest[str(i) + "-0"] = 0.5
    rDictTest[str(i) + "-1"] = 0.5

for key, item in rDict.items():
    rDictTest[key] = item

In [7]:
testMatrix = createFeautre(testData, rDictTest, pDict, mDict, cDict)

In [8]:
# Logistic
logisticRegr = LogisticRegression()
logisticRegr.fit(trainMatrix, labelArray)
predictLogistic = logisticRegr.predict_proba(testMatrix)
print(sorted([e[1] for e in predictLogistic]))

[0.003359194899882091, 0.004215578066175158, 0.004895966481894731, 0.004895966481894731, 0.005852541450080482, 0.007314026183769352, 0.007314026183769352, 0.007383078644085026, 0.007482025131922672, 0.008122692106583052, 0.00818496192531306, 0.008263282332645401, 0.008285215075287627, 0.008342026364471858, 0.008342026364471858, 0.008379499900696337, 0.008426657616758402, 0.008430414561945442, 0.008455586291888039, 0.008519627407189699, 0.008905047052899814, 0.00898158980029523, 0.009063097287609937, 0.00915405205504303, 0.00958812030983363, 0.009589510649050434, 0.009775423963151116, 0.009871410474381287, 0.00997691553890884, 0.010289667351668346, 0.010310344535020341, 0.010317657775094654, 0.010461690989673407, 0.010628091661791163, 0.01065538523005193, 0.010682588601220256, 0.010983233547105339, 0.011199136692517717, 0.011241140590940843, 0.01133696622845594, 0.011384105224836192, 0.011439301875410843, 0.011509838628242388, 0.011767074867158814, 0.012113510795171813, 0.01218732994856

In [9]:
# Gauss
clf = GaussianNB()
clf.fit(trainMatrix, labelArray)
predictGauss = clf.predict_proba(testMatrix)
print(sorted([e[1] for e in predictGauss]))


[1.8362168413228434e-35, 1.8362168413228434e-35, 6.725756250103018e-13, 0.4083523261455808, 0.4089340056719221, 0.4089340056719221, 0.40909445705515507, 0.40930833522512045, 0.40995180665898995, 0.41095075212597815, 0.4115708489388554, 0.4118802842018002, 0.4123634335528198, 0.41243124660816066, 0.41248558812561414, 0.41248558812561414, 0.4126390191838207, 0.41297815663123266, 0.41318240346092827, 0.41318240346092827, 0.41358239025728444, 0.41360870197320054, 0.41363712935302477, 0.41367315752014866, 0.4136793429034764, 0.4139929174107453, 0.4139929174107453, 0.4144792382141833, 0.4144825271278474, 0.41453217258526676, 0.4145782771009867, 0.41462314099669373, 0.41464461122639656, 0.41483834265901937, 0.41483834265901937, 0.4148751516831074, 0.4151944253371369, 0.4152917275546737, 0.41539207845957193, 0.41556064203081544, 0.4155703740402243, 0.41572811743450044, 0.41573367885248746, 0.41583618109805826, 0.4158748339187143, 0.4158752905643157, 0.41609525828579297, 0.41610646999987433, 0.

In [10]:
# RF
rf = RandomForestClassifier()
rf.fit(trainMatrix, labelArray)
predictRF = rf.predict_proba(testMatrix)
print(sorted([e[1] for e in predictRF]))


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [11]:
knc = KNeighborsClassifier()
knc.fit(trainMatrix, labelArray)
predictknc = knc.predict_proba(testMatrix)
print(sorted([e[1] for e in predictknc]))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [13]:

sgd = SGDClassifier(loss='log')
sgd.fit(trainMatrix, labelArray)
predictsgd = sgd.predict_proba(testMatrix)
print(sorted([e[1] for e in predictsgd]))

/usr/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

/usr/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [40]:
dtc = DecisionTreeClassifier()
dtc.fit(trainMatrix, labelArray)
predictdtc = dtc.predict_proba(testMatrix)


In [44]:
predictdtc2 = [ 0.0 if e[1] <0.4 else e[1]  for e in predictdtc]
predictdtc3 = [ 1.0 if e[1] >0.7 else e[1]  for e in predictdtc2]

In [45]:
myData = [["orderItemID", "returnShipment"]] + [[str(i+1), str(e)] for i,e in enumerate(predictdtc3)]

myFile = open('predictDTClassifier.csv', 'w')
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(myData)
     
print("Writing complete")


Writing complete


In [356]:
# -------------------------------------------------

In [12]:
sDict = {}    
grouped_df = data.groupby(['state', 'price'])['state', 'price']

for key, item in grouped_df:
    sDict[str(key[0])] = key[1]


for key, item in pDict.items():
    if item[1] == 1:
        pDict[key] = item[0]
    if item[1] != 1:
        pDict[key] = item[0]/(item[1]-1)

TypeError: 'int' object is not subscriptable

In [11]:
grouped_df = data.groupby(['itemID','returnShipment'])['itemID','returnShipment']
for key, item in grouped_df:
    print(len(grouped_df.get_group(key)), "\n\n")

335 


431 


235 


154 


355 


163 


114 


198 


417 


413 


372 


358 


118 


110 


292 


137 


75 


208 


26 


47 


122 


28 


28 


29 


34 


12 


189 


249 


178 


171 


13 


6 


226 


308 


32 


12 


275 


155 


75 


32 


17 


49 


1221 


987 


28 


35 


59 


70 


436 


350 


168 


208 


224 


194 


369 


244 


14 


133 


118 


113 


275 


72 


2314 


1259 


873 


302 


31 


34 


20 


30 


447 


125 


191 


263 


151 


223 


152 


364 


113 


26 


254 


273 


123 


186 


48 


103 


120 


18 


237 


235 


231 


95 


154 


284 


64 


67 


80 


271 


38 


32 


409 


392 


165 


222 


275 


202 


145 


55 


427 


205 


93 


272 


1350 


745 


349 


332 


482 


149 


10 


116 


273 


188 


682 


192 


219 


154 


116 


106 


222 


131 


75 


29 


134 


410 


775 


417 


792 


500 


388 


335 


397 


621 


261 


149 


69 


70 


316 


244 


51 

39 


17 


31 


10 


11 


44 


22 


38 


39 


1 


18 


10 


3 


13 


16 


2 


5 


1 


18 


5 


11 


11 


1 


3 


9 


13 


23 


56 


5 


7 


38 


38 


18 


35 


12 


4 


3 


2 


61 


1 


10 


9 


5 


37 


32 


3 


3 


10 


6 


5 


2 


31 


60 


5 


4 


27 


6 


12 


3 


1 


9 


2 


2 


4 


3 


31 


12 


18 


12 


1 


29 


34 


1 


11 


8 


19 


14 


5 


3 


3 


1 


7 


4 


27 


50 


1 


12 


8 


19 


31 


81 


14 


8 


1 


3 


35 


80 


27 


45 


2 


2 


20 


18 


21 


2 


7 


1 


8 


120 


63 


16 


12 


37 


56 


7 


17 


5 


12 


5 


8 


2 


1 


1 


36 


41 


3 


9 


9 


11 


2 


3 


10 


18 


18 


1 


30 


20 


16 


12 


38 


29 


48 


37 


1 


1 


42 


46 


1 


33 


22 


7 


6 


2 


1 


13 


14 


117 


50 


35 


17 


1 


13 


18 


13 


1 


8 


9 


1 


1 


4 


2 


50 


58 


1 


1 


17 


9 


9 


1 


7 


1 



57 


84 


119 


1131 


1172 


72 


98 


95 


118 


38 


46 


21 


31 


99 


70 


155 


250 


12 


27 


178 


264 


40 


13 


283 


162 


139 


170 


178 


72 


367 


260 


177 


203 


318 


404 


611 


494 


303 


573 


101 


192 


127 


151 


48 


58 


873 


424 


200 


59 


757 


836 


142 


47 


46 


37 


161 


150 


218 


306 


280 


230 


91 


285 


271 


310 


284 


279 


144 


189 


312 


464 


193 


177 


98 


263 


41 


31 


17 


29 


195 


191 


45 


22 


305 


303 


39 


36 


39 


17 


41 


23 


158 


199 


790 


361 


61 


80 


47 


79 


60 


107 


442 


493 


204 


363 


51 


69 


69 


63 


142 


154 


331 


290 


71 


64 


12 


20 


515 


404 


37 


44 


450 


353 


115 


434 


39 


30 


314 


564 


248 


324 


34 


12 


81 


62 


109 


100 


109 


92 


74 


110 


37 


61 


439 


223 


43 


40 


17 


25 


35 


35 


195 


1

138 


251 


437 


118 


162 


75 


123 


251 


193 


62 


136 


136 


200 


522 


198 


392 


307 


217 


226 


433 


633 


122 


235 


51 


55 


169 


328 


61 


105 


121 


242 


1093 


701 


126 


248 


10 


55 


251 


80 


226 


451 


325 


419 


143 


238 


418 


113 


78 


106 


56 


27 


192 


64 


668 


612 


386 


559 


5 


46 


93 


115 


301 


283 


36 


11 


74 


141 


5 


11 


21 


58 


12 


6 


309 


506 


88 


107 


349 


313 


132 


253 


334 


336 


140 


37 


460 


341 


66 


175 


257 


287 


74 


164 


53 


318 


274 


151 


32 


56 


257 


355 


430 


50 


81 


7 


60 


77 


340 


261 


595 


487 


232 


342 


65 


78 


248 


136 


139 


108 


61 


129 


402 


568 


113 


95 


128 


345 


371 


466 


101 


83 


5 


29 


74 


98 


41 


88 


292 


201 


69 


65 


43 


95 


45 


73 


237 


357 


16 


11 


44 


43 


30 


In [13]:
max(testData["customerID"])

91911